In [18]:
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score
from sklearn.metrics import auc as calculate_auc
from sklearn.datasets import load_breast_cancer

import matplotlib.pyplot as plt
import seaborn as sns

from aggmap import AggMap, AggModel

np.random.seed(666) #just for reaptable results

def prc_auc_score(y_true, y_score):
    precision, recall, threshold  = precision_recall_curve(y_true, y_score) #PRC_AUC
    auc = calculate_auc(recall, precision)
    return auc

In [19]:
data = load_breast_cancer()
dfx = pd.DataFrame(data.data, columns= data.feature_names)
y = pd.Series(data.target).map({0:'malignant', 1:'benign'})
dfy = pd.get_dummies(y) 
X = dfx.values.astype(float)
Y = dfy.values.astype(float)

In [20]:
mp = AggMap(dfx, metric = 'correlation')
mp.fit(cluster_channels = 5, verbose = 0)

2020-10-18 17:08:58,496 - INFO - [bidd-aggmap] - Calculating distance ...
2020-10-18 17:08:58,497 - INFO - [bidd-aggmap] - the number of process is 16


100%|##########| 30/30 [00:00<00:00, 643.72it/s]


2020-10-18 17:09:00,122 - INFO - [bidd-aggmap] - applying hierarchical clustering to obtain group information ...
2020-10-18 17:09:00,193 - INFO - [bidd-aggmap] - Applying grid feature map(assignment), this may take several minutes(1~30 min)
2020-10-18 17:09:00,202 - INFO - [bidd-aggmap] - Finished


## 5-fold cross validation and repeat 5 times 

In [21]:
outer_fold = 5

run_all = []
for repeat_seed in [8, 16, 32, 64, 128]: #5 repeats random seeds
    
    outer = KFold(n_splits = outer_fold, shuffle = True, random_state = repeat_seed)
    outer_idx = outer.split(range(len(Y)), Y[:,0])

    for i, idx in enumerate(outer_idx):
        
        fold_num = "fold_%s" % str(i).zfill(2) 
        
        train_idx, test_idx = idx
        
        testY = Y[test_idx]
        testx = X[test_idx]
        
        trainx = X[train_idx]
        trainY = Y[train_idx]

        trainX = mp.batch_transform(trainx, scale_method = 'standard')
        testX = mp.batch_transform(testx, scale_method = 'standard')
        
        print("\n input train and test X shape is %s, %s " % (trainX.shape,  testX.shape))
        
        clf = AggModel.MultiClassEstimator(epochs = 100, gpuid = 3, conv1_kernel_size = 11, batch_size = 8, verbose = 0)
        clf.fit(trainX, trainY)

        y_true = testY[:,0]
        y_pred = clf.predict(testX)[:,0]
        y_score = clf.predict_proba(testX)[:,0]

        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        
        acc = (tp + tn) / sum([tn, fp, fn, tp])
        
        sensitivity = tp / sum([tp, fn])
        specificity = tn / sum([tn, fp])
        
        prc_auc = prc_auc_score(y_true, y_score)
        roc_auc = roc_auc_score(y_true, y_score)
        
        precision = tp / sum([tp, fp])
        recall =  tp / sum([tp, fn]) #equals to sensitivity
        
        
        res = {'fold': fold_num,
               'repeat_seed':repeat_seed,
               
               'accuracy':acc, 
               
               'prc_auc':prc_auc, 
               'roc_auc':roc_auc,

               'sensitivity': sensitivity, 
               'specificity': specificity,
               
               'precision':precision,
               'recall':recall,
              }
        
        run_all.append(res)

100%|##########| 114/114 [00:00<00:00, 1173.89it/s]



 input train and test X shape is (455, 6, 5, 5), (114, 6, 5, 5) 
MultiClassEstimator(batch_size=8, epochs=100, gpuid='3')


100%|##########| 114/114 [00:00<00:00, 1157.82it/s]



 input train and test X shape is (455, 6, 5, 5), (114, 6, 5, 5) 
MultiClassEstimator(batch_size=8, epochs=100, gpuid='3')


100%|##########| 114/114 [00:00<00:00, 1075.66it/s]


...


 input train and test X shape is (455, 6, 5, 5), (114, 6, 5, 5) 
MultiClassEstimator(batch_size=8, epochs=100, gpuid='3')


100%|##########| 114/114 [00:00<00:00, 1296.54it/s]



 input train and test X shape is (455, 6, 5, 5), (114, 6, 5, 5) 
MultiClassEstimator(batch_size=8, epochs=100, gpuid='3')


100%|##########| 113/113 [00:00<00:00, 1105.69it/s]



 input train and test X shape is (456, 6, 5, 5), (113, 6, 5, 5) 
MultiClassEstimator(batch_size=8, epochs=100, gpuid='3')


In [22]:
clf._model.count_params()

323138

In [23]:
df = pd.DataFrame(run_all)
df.to_excel('results.xlsx')
df

,fold,repeat_seed,accuracy,prc_auc,roc_auc,sensitivity,specificity,precision,recall
0,fold_00,8,0.973684,0.998427,0.997762,0.970588,0.978261,0.985075,0.970588
1,fold_01,8,0.964912,0.998809,0.998016,0.958333,0.976190,0.985714,0.958333
2,fold_02,8,0.982456,0.987934,0.985075,1.000000,0.957447,0.971014,1.000000
3,fold_03,8,0.973684,0.987289,0.983696,0.985294,0.956522,0.971014,0.985294
4,fold_04,8,0.964602,0.999112,0.997640,0.987805,0.903226,0.964286,0.987805
5,fold_00,16,0.956140,0.994579,0.990812,0.972603,0.926829,0.959459,0.972603
6,fold_01,16,0.991228,1.000000,1.000000,0.985714,1.000000,1.000000,0.985714
7,fold_02,16,0.973684,0.998249,0.996537,0.986842,0.947368,0.974026,0.986842
8,fold_03,16,0.982456,0.998860,0.998412,0.985075,0.978723,0.985075,0.985075
9,fold_04,16,0.955752,0.987508,0.981556,0.957746,0.952381,0.971429,0.957746


In [24]:
results = df.groupby(['repeat_seed']).apply(np.mean)
results

,repeat_seed,accuracy,prc_auc,roc_auc,sensitivity,specificity,precision,recall
repeat_seed,,,,,,,,
8,8.0,0.971868,0.994314,0.992438,0.980404,0.954329,0.975421,0.980404
16,16.0,0.971852,0.995839,0.993463,0.977596,0.961060,0.977998,0.977596
32,32.0,0.977177,0.996553,0.994668,0.982761,0.968482,0.980029,0.982761
64,64.0,0.970160,0.996466,0.993990,0.977979,0.953450,0.976220,0.977979
128,128.0,0.968406,0.995928,0.992478,0.980313,0.946251,0.969657,0.980313


In [25]:
results.mean().round(3)

repeat_seed    49.600
accuracy        0.972
prc_auc         0.996
roc_auc         0.993
sensitivity     0.980
specificity     0.957
precision       0.976
recall          0.980
dtype: float64

In [26]:
results.mean().round(3)

repeat_seed    49.600
accuracy        0.972
prc_auc         0.996
roc_auc         0.993
sensitivity     0.980
specificity     0.957
precision       0.976
recall          0.980
dtype: float64

In [27]:
results.std().round(3)

repeat_seed    48.793
accuracy        0.003
prc_auc         0.001
roc_auc         0.001
sensitivity     0.002
specificity     0.008
precision       0.004
recall          0.002
dtype: float64